In [37]:
import pandas

# Question
人脸特征怎么得到的 openai clip √
各种信息都有 
是不是还需要一个本来耦合的信息 √
onlineEvalOnBatch的实现方式


可以不用one clip
768可能比one 更合适
最合适的是RAF（表情分类loss，关键点loss，没有idloss、分类，人脸特征重构loss是有的。）
768 表情分类的loss没有，其他的都有
# 为什么要给我one clip
为了见到每一个人，做测试
只有关键点和重构loss
# 短期目标
*从openai clip 算一个特征 做表情特征检索更靠谱。*
# 长期目标
从图片出发的128位特征

出来的prompt可能不全是表情prompt，embedding进行清理


In [38]:
# 耦合特征 
# lda_model 出来的是人脸识别的特征 其实也带表情
# one_clip = pandas.read_parquet("one_clip_per_identity.parquet", engine="pyarrow")

# 处理celeb_name变为数字
# celeb_name = one_clip["celeb_name"].unique()
# celeb_name_dict = {name: i for i, name in enumerate(celeb_name)}
# one_clip["celeb_name"] = one_clip["celeb_name"].map(celeb_name_dict)
# one_clip

In [39]:
# 固定numpy torch 生成的随机数
import numpy as np
import torch

seed = 129

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [40]:
# 关于表情耦合特征
clip = pandas.read_parquet("RAF_clip_openai.parquet", engine="pyarrow")

# image_name	label	feature
# 处理label=label-1
clip["label"] = clip["label"] - 1
# 采样70%的数据作为训练集
RAF_clip_train = clip.sample(frac=0.7)
# 剩下的数据作为测试集
RAF_clip_test = clip.drop(RAF_clip_train.index)


In [41]:
# 加载768数据集
clip_768 = pandas.read_parquet("top_768_identity.parquet", engine="pyarrow")

# 处理celeb_name变为数字
celeb_name = clip_768["celeb_name"].unique()
celeb_name_dict = {name: i for i, name in enumerate(celeb_name)}
clip_768["label"] = clip_768["celeb_name"].map(celeb_name_dict)

# 采样70%的数据作为训练集
clip_768_train = clip_768.sample(frac=0.7)
# 剩下的数据作为测试集
clip_768_test = clip_768.drop(clip_768_train.index)

In [42]:
import numpy as np
import torch


class DecoupleDataSet(torch.utils.data.Dataset):
    def __init__(self, clip: pandas.DataFrame):
        self.feature = clip["feature"].tolist()
        self.label = clip["label"].tolist()

    def __len__(self):

        return len(self.feature)

    def __getitem__(self, idx):
        # 模型输入
        # x, identity_label=None, expression_label=None
        return self.feature[idx], self.label[idx]

    def collate_fn(self, batch: list[tuple[np.ndarray, int, np.ndarray, int]]):
        # 从batch中提取出数据和标签
        feature, label = zip(*batch)
        # 转换为tensor
        feature = torch.tensor(feature, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)
        return feature, label


class Dataloader(torch.utils.data.DataLoader):
    def __init__(self, dataset: DecoupleDataSet, batch_size=256, **kwargs):
        super(Dataloader, self).__init__(
            dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, **kwargs
        )

In [43]:
from torch import nn


class Loss(nn.Module):
    def __init__(
        self,
        #  identity_loss: Literal["l1", "l2"] = "l2"
    ):
        super(Loss, self).__init__()
        # if identity_loss not in ["l1", "l2"]:
        #     raise ValueError("identity_loss must be either 'l1' or 'l2'")

        # self.identity_loss = nn.L1Loss() if identity_loss == "l1" else nn.MSELoss()
        self.identity_loss = nn.CrossEntropyLoss()
        self.expression_loss_1 = nn.CrossEntropyLoss()
        self.expression_loss_2 = nn.L1Loss()
        self.decouple_loss = self.onlineEvalOnBatch
        # L_Decouple =  E_{sample in batch}( ( f_i - E[f_i] ) * (g_j - E[g_j] ) ) （在batch上计算）有5000个这样的值

    def forward(
        self,
        identity_features,
        expression_features,
        identity_labels=None,
        expression_labels=None,
    ):

        loss = 0
        loss_count = 0

        if identity_labels is not None:
            identity_labels = identity_labels.cuda()
            identity_loss = self.identity_loss(identity_features, identity_labels)
            loss += identity_loss
            loss_count += 1

        if expression_labels is not None:
            expression_labels = expression_labels.cuda()
            expression_loss_1 = self.expression_loss_1(
                expression_features, expression_labels
            )

            loss += expression_loss_1
            loss_count += 1

        decouple_loss = self.decouple_loss(identity_features, expression_features)
        loss += decouple_loss
        loss_count += 1

        return loss / loss_count

    # 要除
    def onlineEvalOnBatch(
        self, identity_features: torch.tensor, expression_features: torch.tensor
    ):
        device = identity_features.device
        bs, dim_i = identity_features.shape
        bs, dim_e = expression_features.shape

        E_i = torch.zeros(dim_i).to(device)
        E_e = torch.zeros(dim_e).to(device)

        avg_on_batch_i = torch.mean(identity_features, dim=0)
        avg_on_batch_e = torch.mean(expression_features, dim=0)

        decouple_loss = 0

        for i in range(dim_i):
            length = i + 1
            new_avg_on_length = torch.sum(avg_on_batch_i[:length]) / length
            E_i[i] = new_avg_on_length * 0.1 + E_i[i - 1] * 0.9

        for i in range(dim_e):
            length = i + 1
            new_avg_on_length = torch.sum(avg_on_batch_e[:length]) / length
            E_e[i] = new_avg_on_length * 0.1 + E_e[i - 1] * 0.9

        # [dim_i,1] @ [1,dim_e] = [dim_i,dim_e]
        decouple_loss = torch.sum(
            (avg_on_batch_i - E_i).unsqueeze(1) @ (avg_on_batch_e - E_e).unsqueeze(0)
        )

        return decouple_loss / dim_i / dim_e

In [44]:
class Decouple(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Decouple, self).__init__()

        # self.out = nn.TransformerEncoderLayer(
        #     d_model=input_dim, nhead=8, dim_feedforward=2048
        # )
        # 多层Transformer
        self.out = nn.Sequential(
            
            nn.TransformerEncoder(
                nn.TransformerEncoderLayer(
                    d_model=input_dim, nhead=8, dim_feedforward=input_dim
                ),
                num_layers=6,
            ),
            nn.ReLU(),
            nn.Linear(input_dim, output_dim),
        )

    def forward(self, x):
        return self.out(x)

In [45]:
from huggingface_hub import hf_hub_download
import pickle


class fooModel(nn.Module):
    def __init__(
        self, identity_features_dim: int = 512, expression_features_dim: int = 256
    ):
        super(fooModel, self).__init__()
        lda_model_path = hf_hub_download(
            repo_id="silk-road/simple-face-recognition",
            filename="lda_openai_clip_model.pkl",
        )

        with open(lda_model_path, "rb") as f:
            self.lda_model = pickle.load(f)

        # 激活函数
        self.relu = nn.ReLU()

        # 解耦函数
        self.fx = Decouple(512, 7)
        self.gx = Decouple(512, 7)

        self.loss = Loss()

        # softmax
        self.softmax = nn.Softmax(dim=1)

        # dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, identity_label=None, expression_label=None):
        x = torch.tensor(self.lda_model.transform(x), dtype=torch.float32)
        x = x.cuda()
        # Normalize
        x = (x - x.mean()) / x.std()

        # 人脸识别特征分离
        identity_features = self.fx(x)
        identity_features = self.softmax(identity_features)
        identity_features = self.dropout(identity_features)

        # 表情识别特征分离
        expression_features = self.gx(x)
        expression_features = self.softmax(expression_features)
        expression_features = self.dropout(expression_features)

        if identity_label is not None or expression_label is not None:
            return self.loss(
                identity_features, expression_features, identity_label, expression_label
            )

        return identity_features, expression_features

In [46]:
from sklearn import random_projection


class RandomProjection(nn.Module):
    def __init__(self):
        super(RandomProjection, self).__init__()
        lda_model_path = hf_hub_download(
            repo_id="silk-road/simple-face-recognition",
            filename="lda_openai_clip_model.pkl",
        )

        with open(lda_model_path, "rb") as f:
            self.lda_model = pickle.load(f)

        self.identity_random_projection = random_projection.GaussianRandomProjection(7)
        self.expression_random_projection = random_projection.GaussianRandomProjection(
            7
        )

    def forward(self, x: torch.tensor) -> tuple[torch.tensor, torch.tensor]:
        x = torch.tensor(self.lda_model.transform(x), dtype=torch.float32)
        
        identity_features: np.array = self.identity_random_projection.fit_transform(x)
        expression_features: np.array = self.expression_random_projection.fit_transform(
            x
        )

        return torch.tensor(identity_features, dtype=torch.float32), torch.tensor(
            expression_features, dtype=torch.float32
        )


# 测试RandomProjection
x = torch.randn(256, 512)
r = RandomProjection()
i, e = r(x)
i.shape, e.shape

(torch.Size([256, 7]), torch.Size([256, 7]))

In [47]:
from tqdm import tqdm


model = fooModel().cuda()
model.train()

# expression
RAF_train_dataset = DecoupleDataSet(RAF_clip_train)
RAF_train_dataloader = Dataloader(RAF_train_dataset, batch_size=256, shuffle=True)

# # identity
# train_dataset_768 = DecoupleDataSet(clip_768_train)
# train_dataloader_768 = Dataloader(train_dataset_768, batch_size=256, shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

# 训练模型
for epoch in range(2):
    t = tqdm(RAF_train_dataloader)
    for batch in t:
        features, expression_label = batch

        loss = model(features, expression_label=expression_label)
        t.set_postfix(loss=loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # t = tqdm(train_dataloader_768)
    # for batch in t:
    #     features, identity_label = batch

    #     loss = model(features, identity_label=identity_label)
    #     t.set_postfix(loss=loss.item())

    #     optimizer.zero_grad()
    #     loss.backward()
    #     optimizer.step()


# 保存模型
# torch.save(model.state_dict(), f"model-{time.time()}.pth")


# expression推理
model.eval()

RAF_test_dataset = DecoupleDataSet(RAF_clip_test)
RAF_test_dataloader = Dataloader(RAF_test_dataset, batch_size=256)

t = tqdm(RAF_test_dataloader)

fx_error = 0
gx_error = 0
fx_random_error = 0
gx_random_error = 0
random_projection = RandomProjection()

for batch in t:
    with torch.no_grad():
        features, expression_label = batch
        identity_features, expression_features = model(features)

        identity_features = identity_features.cpu()
        expression_features = expression_features.cpu()

        fx_error += torch.sum(
            torch.argmax(identity_features, dim=1) != expression_label
        ).item()
        gx_error += torch.sum(
            torch.argmax(expression_features, dim=1) != expression_label
        ).item()

        identity_features_random, expression_features_random = random_projection(
            features
        )

        fx_random_error += torch.sum(
            torch.argmax(identity_features_random, dim=1) != expression_label
        ).item()
        gx_random_error += torch.sum(
            torch.argmax(expression_features_random, dim=1) != expression_label
        ).item()


# 输出正确率(取小数点后两位)
print(
    f"expression acc on RAF by decoupled identity(f(x)): {(1 - fx_error / len(RAF_clip_test))*100:.2f}%"
)
# 输出随机投影正确率(取小数点后两位)
print(
    f"expression acc on RAF by random projection identity: {(1 - fx_random_error / len(RAF_clip_test))*100:.2f}%"
)
print(
    f"expression acc on RAF by decoupled expression(g(x)): {(1 - gx_error / len(RAF_clip_test))*100:.2f}%"
)
# 输出随机投影正确率(取小数点后两位)
print(
    f"expression acc on RAF by random projection expression: {(1 - gx_random_error / len(RAF_clip_test))*100:.2f}%"
)

/workspace/jyh/miniconda3/envs/face/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
100%|██████████| 15/15 [00:10<00:00,  1.41it/s]

expression acc on RAF by decoupled identity(f(x)): 11.03%
expression acc on RAF by random projection identity: 14.15%
expression acc on RAF by decoupled expression(g(x)): 39.58%
expression acc on RAF by random projection expression: 15.00%


In [48]:
from itertools import product


a=[1,2]
b=[1,2,3,4]
for aa,bb in product(enumerate(a),enumerate(b)):
    print(aa,bb)

(0, 1) (0, 1)
(0, 1) (1, 2)
(0, 1) (2, 3)
(0, 1) (3, 4)
(1, 2) (0, 1)
(1, 2) (1, 2)
(1, 2) (2, 3)
(1, 2) (3, 4)
